STEP 1: Setup Environment in Colab

In [1]:
!pip install transformers accelerate bitsandbytes gradio pandas --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 74.3 MB/s eta 0:00:00


STEP 2: Upload Required Files to Colab

In [2]:
from google.colab import files
uploaded = files.upload()


Saving uiu_qa_data.csv to uiu_qa_data.csv


STEP 3: Create retriever.py (Data Lookup Logic)

In [3]:
import pandas as pd

# Load your QA dataset
df = pd.read_csv("uiu_qa_data.csv", encoding="windows-1252")
# Function to get the best-matching answer (simple keyword match for now)
def retrieve_answer(question):
    question_lower = question.lower()
    for _, row in df.iterrows():
        if any(q_word in question_lower for q_word in row["Question"].lower().split()):
            return f"{row['Answer']}\n\n🔗 Source: {row['Source URL']}"
    return "Sorry, I couldn't find an answer to that in our university database."


STEP 4: Load the Meta LLaMA 3 Model

In [ ]:
from huggingface_hub import login
login("Use your actual token")

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,
    trust_remote_code=True,
    use_auth_token=True
)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [26]:
import pandas as pd
import os

def clean_qa_dataset(input_csv_path='uiu_qa_data.csv', output_csv_path='uiu_qa_cleaned.csv'):
    """
    Cleans the UIU QA dataset by removing rows with missing values
    in 'Question', 'Answer', or 'Source URL' columns.

    Args:
        input_csv_path (str): Path to the original CSV dataset.
        output_csv_path (str): Path where the cleaned CSV will be saved.
    """
    print(f"Attempting to load dataset from: {input_csv_path}")
    if not os.path.exists(input_csv_path):
        print(f"Error: The input file '{input_csv_path}' was not found. "
              "Please ensure 'uiu_qa_data.csv' is in the same directory as this script.")
        return

    try:
        # Added encoding='latin1' to handle potential encoding issues
        df = pd.read_csv(input_csv_path, encoding='latin1')
        print("Dataset loaded successfully.")
    except UnicodeDecodeError:
        print(f"UnicodeDecodeError: Could not decode '{input_csv_path}' with 'latin1' encoding. "
              "Trying 'cp1252' encoding.")
        try:
            df = pd.read_csv(input_csv_path, encoding='cp1252')
            print("Dataset loaded successfully with 'cp1252' encoding.")
        except Exception as e:
            print(f"Error reading CSV file with 'cp1252' encoding: {e}")
            return
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return

    # Display initial info
    print("\n--- Original Dataset Info ---")
    df.info()
    print("\nMissing values before cleaning:")
    print(df.isnull().sum())

    # Define critical columns for QA bot functionality
    critical_columns = ['Question', 'Answer', 'Source URL']

    # Drop rows where any of the critical columns have missing values
    df_cleaned = df.dropna(subset=critical_columns)

    # Convert all values in critical columns to string type to prevent errors later
    for col in critical_columns:
        df_cleaned[col] = df_cleaned[col].astype(str)

    print("\n--- Cleaned Dataset Info ---")
    print(f"Original number of rows: {len(df)}")
    print(f"Number of rows after cleaning: {len(df_cleaned)}")
    print("\nMissing values after cleaning:")
    print(df_cleaned.isnull().sum()) # Should show 0 for critical columns

    # Save the cleaned DataFrame to a new CSV file
    try:
        df_cleaned.to_csv(output_csv_path, index=False, encoding='utf-8') # Save as UTF-8 for consistency
        print(f"\nCleaned dataset saved successfully to: {output_csv_path}")
        print(f"You can now find '{output_csv_path}' in the same directory where you run this script.")
    except Exception as e:
        print(f"Error saving cleaned CSV file: {e}")

if __name__ == "__main__":
    # Ensure uiu_qa_data.csv is in the same directory as this script
    clean_qa_dataset()


Attempting to load dataset from: uiu_qa_data.csv
Dataset loaded successfully.

--- Original Dataset Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1493 entries, 0 to 1492
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Question    1461 non-null   object
 1   Answer      1490 non-null   object
 2   Source URL  1460 non-null   object
dtypes: object(3)
memory usage: 35.1+ KB

Missing values before cleaning:
Question      32
Answer         3
Source URL    33
dtype: int64

--- Cleaned Dataset Info ---
Original number of rows: 1493
Number of rows after cleaning: 1460

Missing values after cleaning:
Question      0
Answer        0
Source URL    0
dtype: int64

Cleaned dataset saved successfully to: uiu_qa_cleaned.csv
You can now find 'uiu_qa_cleaned.csv' in the same directory where you run this script.


/tmp/ipython-input-26-2910449475.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[col] = df_cleaned[col].astype(str)
/tmp/ipython-input-26-2910449475.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[col] = df_cleaned[col].astype(str)
/tmp/ipython-input-26-2910449475.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

STEP 5: Build Answer Generator (generate_answer)

In [32]:
# STEP 1: Install dependencies (Google Colab only)
!pip install gradio pandas --quiet


In [33]:
# STEP 2: Import modules
import pandas as pd
import difflib
import gradio as gr


In [ ]:
# STEP 3: Load the cleaned dataset
file_path = "uiu_qa_cleaned.csv"  # Make sure this is uploaded in Colab or exists in your folder

try:
    df = pd.read_csv(file_path)
    print(f"Dataset loaded with {len(df)} entries.")
except Exception as e:
    print(f"Failed to load dataset: {e}")


✅ Dataset loaded with 1460 entries.


In [ ]:
# STEP 4: Define answer generator using fuzzy matching

def generate_answer(user_question):
    user_question = user_question.lower().strip()

    # List of all cleaned questions
    question_list = df["Question"].astype(str).tolist()

    # Get best fuzzy match
    best_match = difflib.get_close_matches(user_question, question_list, n=1, cutoff=0.5)

    if best_match:
        matched_row = df[df["Question"] == best_match[0]].iloc[0]
        answer = matched_row["Answer"]
        source = matched_row["Source URL"]
        return f"**Answer:** {answer}\n\n🔗 [Source]({source})"
    else:
        return "Sorry, I couldn't find an exact answer to your question in the UIU QA database."


In [36]:
# STEP 5: Launch the chatbot using Gradio

with gr.Blocks() as uiu_bot:
    gr.Markdown("## 🎓 United International University - Helpline QA Bot 🤖")
    gr.Markdown("Ask anything related to the university. The bot will answer from verified sources.")

    with gr.Row():
        question_input = gr.Textbox(placeholder="Type your university-related question...", label="Your Question")
        answer_output = gr.Markdown()

    ask_button = gr.Button("Get Answer")
    ask_button.click(fn=generate_answer, inputs=question_input, outputs=answer_output)

    # Optional: Add FAQs or About section
    with gr.Accordion("📌 FAQ", open=False):
        gr.Markdown("""
- **Q:** What is the number of programs in UIU?
  **A:** 17 Programs
- **Q:** Where is the UIU Library?
  **A:** Ground Floor, UIU Campus
- **Q:** What is required credit for Bachelor of CSE?
  **A:** 132 Credits
        """)

    with gr.Accordion("ℹ️ About", open=False):
        gr.Markdown("Developed by **Gallic Rooster Team** using Meta LLaMA-backed structured QA and fuzzy retrieval.")

uiu_bot.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c0952609f0138c7b54.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
